In [84]:
#数据读取
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
imgPath=r"G:\ZMZ\数据集\Brarts\Brats测试结果\hgg_BraTS19_CBICA_AUA_1.nii.gz"
imageLoad=nib.load(imgPath)
imageData=imageLoad.get_fdata()
type(imageData)
print(imageData.shape)

(240, 240, 155)


In [79]:
#归一化
def normalize(slice, bottom=99, down=1):
    """
    normalize image with mean and std for regionnonzero,and clip the value into range
    :param slice:
    :param bottom:
    :param down:
    :return:
    """
    b = np.percentile(slice, bottom)
    # print(b)
    t = np.percentile(slice, down)
    # print(t)
    slice = np.clip(slice, t, b)
    # print(slice)
    image_nonzero = slice[np.nonzero(slice)]
    if np.std(slice) == 0 or np.std(image_nonzero) == 0:
        return slice
    else:
        tmp = (slice - np.mean(image_nonzero)) / np.std(image_nonzero)
        # since the range of intensities is between 0 and 5000 ,
        # the min in the normalized slice corresponds to 0 intensity in unnormalized slice
        # the min is replaced with -9 just to keep track of 0 intensities
        # so that we can discard those intensities afterwards when sampling random patches
        tmp[tmp == tmp.min()] = -9
        return tmp
#裁剪    
# def crop_ceter(img,croph,cropw):   
#     #for n_slice in range(img.shape[0]):
#     height,width = img[0].shape 
#     starth = height//2-(croph//2)
#     startw = width//2-(cropw//2)        
#     return img[:,starth:starth+croph,startw:startw+cropw]

#将三维医学图像数据裁剪为某个固定尺寸
def crop_ceter(data,target_size):
    # 计算裁剪的起始位置和结束位置
    start_index = [(data.shape[i] - target_size[i]) // 2 for i in range(3)]
    end_index = [start_index[i] + target_size[i] for i in range(3)]
    # 裁剪原始医学图像数据
    cropped_image = data[start_index[0]:end_index[0], start_index[1]:end_index[1], start_index[2]:end_index[2]]
    
    return cropped_image

#将三维医学图像数据填充为某个固定尺寸
def pad_volume_to_shape(volume, shape):
    assert len(volume.shape) == len(shape), "Input volume and target shape have different dimensions"

    padded_volume = np.zeros(shape)
    diff = np.asarray(shape) - np.asarray(volume.shape)
    assert (diff >= 0).all(), "Target shape is smaller than input volume shape"

    pad_before = diff // 2
    pad_after = diff - pad_before

    slices = tuple(slice(pb, shape[i] - pa) for i, (pb, pa) in enumerate(zip(pad_before, pad_after)))
    padded_volume[slices] = volume

    return padded_volume

#实现将三维医学图像数据中的某个维度进行填充
def pad_3d_volume_along_axis(volume, axis, target_size):
    """
    在三维体积数据的某个轴上进行填充
    :param volume: 三维体积数据,numpy数组,形状为(d, h, w)
    :param axis: 希望进行填充的轴,可以是0, 1, 2,分别代表深度、高度、宽度
    :param target_size: 目标大小,int类型
    :return: 填充后的三维体积数据,numpy数组,形状为(d', h', w')
    """
    size = volume.shape
    if size[axis] > target_size:
        raise ValueError("The target size should be larger than the current size along the chosen axis.")
    pad_size = target_size - size[axis]
    pad_width = [(0, 0), (0, 0), (0, 0)]
    pad_width[axis] = (pad_size // 2, pad_size - pad_size // 2)
    return np.pad(volume, pad_width, mode='constant', constant_values=0)

#数据标签集掩码处理
def load_label_t1w(mask_path,t1w_path,label_path):
    t1w_data = nib.load(t1w_path).get_fdata()
    mask_data = nib.load(mask_path).get_fdata()
    mask_np = np.array(mask_data)
    label_data = nib.load(label_path).get_fdata()
    label_np = np.array(label_data)
    print(label_np.dtype)
    label_np[label_np==4] = 0
    label_np[label_np==5] = 0
    label_np[label_np==6] = 0
    t1w_mask = np.multiply(t1w_data,mask_np)
    print(np.unique(label_np))
    return t1w_mask,label_np




"import SimpleITK as sitk\n\n# 加载原始nii文件\nimage = sitk.ReadImage('path/to/image.nii')\n\n# 将高度维度填充至64\nimage_data = sitk.GetArrayFromImage(image)\npadded_data = pad_3d_volume_along_axis(image_data, axis=1, target_size=64)\n\n# 将填充后的数据保存为新的nii文件\npadded_image = sitk.GetImageFromArray(padded_data)\npadded_image.CopyInformation(image)\nsitk.WriteImage(padded_image, 'path/to/padded_image.nii')"

In [82]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
imgPath=r"E:\DBB\sub-0415\dt-neuro-anat-t1w.id-61b283327de472d3390c19b4\t1.nii.gz"
mask_path=r"E:\DBB\sub-0415\dt-neuro-mask.id-61b282c552911e175a11ee49\mask.nii.gz"
label_path=r"E:\DBB\sub-0415\dt-neuro-parcellation-volume.id-61b282b852911e175a11ec74\parc.nii.gz"
t1w_mask,label_np=load_label_t1w(mask_path,imgPath,label_path)
print(t1w_mask.shape)
print(label_np.shape)
imageData=normalize(imageData)
target_size = [160,160,160]
data=crop_ceter(imageData,target_size)#数据裁剪
mask=crop_ceter(label_np,target_size)
print(data.shape)
print(mask.shape)
padded_data = pad_3d_volume_along_axis(data, axis=0, target_size=200)#对某一维数据进行填充
print(padded_data.shape)



float64
[0. 1. 2. 3.]
(182, 218, 182)
(182, 218, 182)
(160, 160, 160)
(160, 160, 160)
(200, 160, 160)


In [75]:
#数据分块
BLOCKSIZE=(16, 160, 160)#数据分块大小
patch_block_size = BLOCKSIZE#BLOCKSIZE = (16, 160, 160) #每个分块的大小
numberxy = patch_block_size[1]#160
numberz = 8#patch_block_size[0]
width = np.shape(data)[1]#160
height = np.shape(data)[2]#160
imagez = np.shape(data)[0]#160
block_width = np.array(patch_block_size)[1]#160
block_height = np.array(patch_block_size)[2]#160
blockz = np.array(patch_block_size)[0]#16
stridewidth = (width - block_width) // numberxy#0
strideheight = (height - block_height) // numberxy#0
stridez = (imagez - blockz) // numberz#18
print(stridez)
step_width = width - (stridewidth * numberxy + block_width)#0
step_width = step_width // 2#0
print(step_width)
step_height = height - (strideheight * numberxy + block_height)#0
step_height = step_height // 2
print(step_height)
step_z = imagez - (stridez * numberz + blockz)
step_z = step_z // 2
print('step_z',step_z)#0
hr_samples_t1_list = []
# hr_samples_flair_list = []
# hr_samples_t1ce_list = []
# hr_samples_t2_list = []
hr_mask_samples_list = []
patchnum = []
for z in range(step_z, numberz * (stridez + 1) + step_z, numberz):
    for x in range(step_width, numberxy * (stridewidth + 1) + step_width, numberxy):
        for y in range(step_height, numberxy * (strideheight + 1) + step_height, numberxy):
            if np.max(mask[z:z + blockz, x:x + block_width, y:y + block_height]) != 0:#这里要判断这个分块对应的mask标签是不是背景
                # print("切%d"%z)
                patchnum.append(z)
                #这里注释掉，是因为这个数据只有1个T1,不是4种模态
                hr_samples_t1_list.append(data[z:z + blockz, x:x + block_width, y:y + block_height])
                hr_mask_samples_list.append(mask[z:z + blockz, x:x + block_width, y:y + block_height])
                # hr_samples_flair_list.append(flair_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                # hr_samples_t1ce_list.append(t1ce_crop[z:z + blockz, x:x + block_width, y:y + block_height])
                # hr_samples_t2_list.append(t2_crop[z:z + blockz, x:x + block_width, y:y + block_height])
#这里跟上边相同，也是只有一种模态
samples_t1 = np.array(hr_samples_t1_list).reshape((len(hr_samples_t1_list), blockz, block_width, block_height))
mask_samples = np.array(hr_mask_samples_list).reshape((len(hr_mask_samples_list), blockz, block_width, block_height))
# samples_flair = np.array(hr_samples_flair_list).reshape((len(hr_samples_flair_list), blockz, block_width, block_height))
# samples_t1ce = np.array(hr_samples_t1ce_list).reshape((len(hr_samples_t1ce_list), blockz, block_width, block_height))
# samples_t2 = np.array(hr_samples_t2_list).reshape((len(hr_samples_t2_list), blockz, block_width, block_height))

#这里是分块的个数，以及每个分块的D，H，W
samples, imagez, height, width = np.shape(samples_t1)[0], np.shape(samples_t1)[1], \
                                np.shape(samples_t1)[2], np.shape(samples_t1)[3]
print(samples, imagez, height, width)#19 16 160 160
# 5、合并和保存
for j in range(samples):
    #如果是多模态就要把数据通道数改一下
    fourmodelimagearray = np.zeros((imagez, height, width, 1), np.float)
    #保存路径
    dataSavePath="./output_t1.npy"
    labelSavePath="./output_label.npy"

    # filepath1 = trainImage + "\\" + part + "_" + pathhgg_list[subsetindex] + "_" + str(patchnum[j]) + ".npy"
    # print(filepath1)
    # filepath = trainMask + "\\" + part + "_" + pathhgg_list[subsetindex] + "_" + str(patchnum[j]) + ".npy"
    # print(filepath)

    #这里只有t1，所以只用了t1模态
    t1image = samples_t1[j, :, :, :]
    t1image = t1image.astype(np.float)
    fourmodelimagearray[:, :, :, 0] = t1image

    # flairimage = samples_flair[j, :, :, :]
    # flairimage = flairimage.astype(np.float)
    # fourmodelimagearray[:, :, :, 0] = flairimage
    # t1ceimage = samples_t1ce[j, :, :, :]
    # t1ceimage = t1ceimage.astype(np.float)
    # fourmodelimagearray[:, :, :, 2] = t1ceimage
    # t2image = samples_t2[j, :, :, :]
    # t2image = t2image.astype(np.float)
    # fourmodelimagearray[:, :, :, 3] = t2image
    print(fourmodelimagearray.shape)
    
    #保存数据为npy文件
    np.save(dataSavePath, fourmodelimagearray)
    
    #创建一个空的3通道的用于放标签的数组
    wt_tc_etMaskArray = np.zeros((imagez, height, width, 3), np.uint8)
    mask_one_sample = mask_samples[j, :, :, :]
    C1_Label = mask_one_sample.copy()
    C1_Label[mask_one_sample==1]=1
    C2_Label = mask_one_sample.copy()
    C2_Label[mask_one_sample==2]=2
    C3_Label = mask_one_sample.copy()
    C3_Label[mask_one_sample==3]=3
    wt_tc_etMaskArray[:, :, :, 0] = C1_Label
    wt_tc_etMaskArray[:, :, :, 1] = C2_Label
    wt_tc_etMaskArray[:, :, :, 2] = C3_Label
    
    #不是Brats数据集一般用不到
    # WT_Label[mask_one_sample == 1] = 1.
    # WT_Label[mask_one_sample == 2] = 1.
    # WT_Label[mask_one_sample == 4] = 1.
    # TC_Label = mask_one_sample.copy()
    # TC_Label[mask_one_sample == 1] = 1.
    # TC_Label[mask_one_sample == 2] = 0.
    # TC_Label[mask_one_sample == 4] = 1.
    # ET_Label = mask_one_sample.copy()
    # ET_Label[mask_one_sample == 1] = 0.
    # ET_Label[mask_one_sample == 2] = 0.
    # ET_Label[mask_one_sample == 4] = 1.
    # wt_tc_etMaskArray[:, :, :, 0] = WT_Label
    # wt_tc_etMaskArray[:, :, :, 1] = TC_Label
    # wt_tc_etMaskArray[:, :, :, 2] = ET_Label

    np.save(labelSavePath, wt_tc_etMaskArray)
print("Done!")

18
0
0
step_z 0
19 16 160 160
(16, 160, 160, 1)
(16, 160, 160, 1)
(16, 160, 160, 1)
(16, 160, 160, 1)
(16, 160, 160, 1)
(16, 160, 160, 1)
(16, 160, 160, 1)


C:\Users\zhao'ming'zhang\AppData\Local\Temp\ipykernel_22320\1699466307.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fourmodelimagearray = np.zeros((imagez, height, width, 1), np.float)
C:\Users\zhao'ming'zhang\AppData\Local\Temp\ipykernel_22320\1699466307.py:68: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  t1image = t1image.astyp

(16, 160, 160, 1)
(16, 160, 160, 1)
(16, 160, 160, 1)
(16, 160, 160, 1)
(16, 160, 160, 1)
(16, 160, 160, 1)
(16, 160, 160, 1)
(16, 160, 160, 1)
(16, 160, 160, 1)
(16, 160, 160, 1)
(16, 160, 160, 1)
(16, 160, 160, 1)
Done!


In [77]:
data=np.load(r'./output_t1.npy')
print(data.shape)

(16, 160, 160, 1)


In [19]:
import torch
import os
import torch.utils.data as data

In [25]:
def load_label_t1w(mask_path,t1w_path,label_path):
    t1w_data = nib.load(t1w_path).get_fdata()
    mask_data = nib.load(mask_path).get_fdata()
    mask_np = np.array(mask_data)
    label_data = nib.load(label_path).get_fdata()
    label_np = np.array(label_data)
    print(label_np.dtype)
    label_np[label_np==4] = 0
    label_np[label_np==5] = 0
    label_np[label_np==6] = 0
    t1w_mask = np.multiply(t1w_data,mask_np)
    print(np.unique(label_np))
    return t1w_mask,label_np

class Load_data_dbb(data.Dataset):
    def __init__(self,mask_paths,t1w_paths,label_paths):
        super(Load_data_dbb, self).__init__()
        # self.load_label_t1w = load_label_t1w
        self.mask_paths = mask_paths
        self.t1w_paths = t1w_paths
        self.label_paths = label_paths

    def __getitem__(self,idx):
        image , mask = load_label_t1w(self.mask_paths[idx], self.t1w_paths[idx], self.label_paths[idx])
        image_tensor = torch.from_numpy(image).float()
        mask_tensor = torch.from_numpy(mask).long()
        return image_tensor ,mask_tensor

    def __len__(self):
        return len(self.mask_paths)

def read_paths(root_dir):
    subjects = os.listdir(root_dir)
    t1w_paths = []
    mask_paths = []
    label_paths = []
    for subject in subjects:
        subject_dir = os.path.join(root_dir,subject)
        subject_files = os.listdir(subject_dir)
        for file_dir in subject_files:
            if('t1w' in file_dir):
                t1w_path = subject_dir + '/' + file_dir + '/t1.nii.gz'
                t1w_paths.append(t1w_path)
            elif('mask' in file_dir):
                mask_path = subject_dir + '/' + file_dir + '/mask.nii.gz'
                mask_paths.append(mask_path)
            elif('parcellation' in file_dir):
                label_path= subject_dir + '/' + file_dir + '/parc.nii.gz'
                label_paths.append(label_path)
    # print(mask_paths)
    return t1w_paths,mask_paths,label_paths
root_dir=r'E:\DBB\DBB_SUB'
t1w_paths,mask_paths,label_paths=read_paths(root_dir)
train_dataset = Load_data_dbb(t1w_paths,mask_paths,label_paths)
print(len(train_dataset))



6
